In [1]:
import os
import json

pivot_rules = ["dantzig", "steepest_edge", "random"]
categories = ["balanced_problems", "tall_problems", "wide_problems"]
problem_sets = ["one_phase_only", "two_phases"]

# Function to load data from JSON files, return empty dictionary if file does not exist
def load_json_file(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    else:
        print(f"File not found: {file_path}")
        return {}

results = {category: {problem_set: {rule: [] for rule in pivot_rules} for problem_set in problem_sets} for category in categories}
balanced_results = {rule: {"optimal": [], "infeasible": [], "unbounded": []} for rule in pivot_rules}

for category in categories:
    # For one-phase and two-phase problem sets
    for problem_set in problem_sets:
        base_path = os.path.join("problems", f"problems_pivot_rules_{problem_set}", category)
        
        for size_folder in os.listdir(base_path):
            size_folder_path = os.path.join(base_path, size_folder)
            
            if problem_set == "one_phase_only":
                averages_file = os.path.join(size_folder_path, "averages.json")
                data = load_json_file(averages_file)
                
                # Calculate the average times for each pivot rule
                for rule in pivot_rules:
                    if rule in data["total_time"]:
                        results[category][problem_set][rule].append(data["total_time"][rule])

            elif problem_set == "two_phases":
                averages_file = os.path.join(size_folder_path, "averages.json")
                data = load_json_file(averages_file)
                
                for rule in pivot_rules:
                    if rule in data["total_time"]:
                        results[category][problem_set][rule].append(data["total_time"][rule])
                
                if category == "balanced_problems":
                    for status in ["Optimal", "Infeasible", "Unbounded"]:
                        status_file = os.path.join(size_folder_path, f"averages_{status}.json")
                        status_data = load_json_file(status_file)
                        
                        for rule in pivot_rules:
                            if rule in status_data.get("total_time", {}):
                                balanced_results[rule][status.lower()].append(status_data["total_time"][rule])

# Now, compute the averages for each category, problem set, and pivot rule
final_results = {}

for category, problem_data in results.items():
    final_results[category] = {}
    
    for problem_set, rule_data in problem_data.items():
        final_results[category][problem_set] = {}
        
        for rule, time_data in rule_data.items():
            # Calculate the average for each pivot rule, in each problem set
            if len(time_data) > 0:
                final_results[category][problem_set][rule] = sum(time_data) / len(time_data)
            else:
                final_results[category][problem_set][rule] = 0

balanced_avg_times = {}
for rule in pivot_rules:
    balanced_avg_times[rule] = {}
    for status in ["optimal", "infeasible", "unbounded"]:
        if balanced_results[rule][status]:
            balanced_avg_times[rule][status] = sum(balanced_results[rule][status]) / len(balanced_results[rule][status])
        else:
            balanced_avg_times[rule][status] = 0

print("Average times for balanced problems (one-phase and two-phase):")
print(json.dumps(final_results["balanced_problems"], indent=4))

print("\nAverage times for Infeasible/Optimal/Unbounded (balanced problems):")
print(json.dumps(balanced_avg_times, indent=4))


Average times for balanced problems (one-phase and two-phase):
{
    "one_phase_only": {
        "dantzig": 3832.708462190628,
        "steepest_edge": 2299.582421922684,
        "random": 7975.263597917557
    },
    "two_phases": {
        "dantzig": 9085.94040608406,
        "steepest_edge": 3248.824888277054,
        "random": 14019.446242904663
    }
}

Average times for Infeasible/Optimal/Unbounded (balanced problems):
{
    "dantzig": {
        "optimal": 10558.922885433416,
        "infeasible": 8013.370296362026,
        "unbounded": 9873.926392396019
    },
    "steepest_edge": {
        "optimal": 3582.097643018281,
        "infeasible": 2772.3060955972264,
        "unbounded": 3635.6348614033654
    },
    "random": {
        "optimal": 16511.236087878002,
        "infeasible": 13196.03009915041,
        "unbounded": 14471.17527281833
    }
}


In [2]:
# Relative time differences between two-phase and one-phase
time_diff = {}
for rule in pivot_rules:
    time_diff[rule] = {
        "two_phase_vs_one_phase": final_results["balanced_problems"]["two_phases"].get(rule, 1) / final_results["balanced_problems"]["one_phase_only"].get(rule, 1),
        "infeasible_two_phase_vs_one_phase": balanced_avg_times[rule]["infeasible"] / final_results["balanced_problems"]["one_phase_only"].get(rule, 1)
    }

print("Relative Time Differences Between Two-Phase and One-Phase Problems:")
print(json.dumps(time_diff, indent=4))

Relative Time Differences Between Two-Phase and One-Phase Problems:
{
    "dantzig": {
        "two_phase_vs_one_phase": 2.370631759685392,
        "infeasible_two_phase_vs_one_phase": 2.0907852437547243
    },
    "steepest_edge": {
        "two_phase_vs_one_phase": 1.412789060007124,
        "infeasible_two_phase_vs_one_phase": 1.2055693543175103
    },
    "random": {
        "two_phase_vs_one_phase": 1.757866190976474,
        "infeasible_two_phase_vs_one_phase": 1.65461993038024
    }
}


In [3]:
import os
import json

pivot_rules = ["dantzig", "steepest_edge", "random"]
categories = ["balanced_problems", "tall_problems", "wide_problems"]
problem_sets = ["one_phase_only", "two_phases"]

# Function to load data from JSON files, return empty dictionary if file does not exist
def load_json_file(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    else:
        print(f"File not found: {file_path}")
        return {}

results = {category: {problem_set: {rule: [] for rule in pivot_rules} for problem_set in problem_sets} for category in categories}
balanced_results = {rule: {"optimal": [], "infeasible": [], "unbounded": []} for rule in pivot_rules}
second_phase_percentage = {rule: {"optimal": []} for rule in pivot_rules}

for category in categories:
    for problem_set in problem_sets:
        base_path = os.path.join("problems", f"problems_pivot_rules_{problem_set}", category)
        
        for size_folder in os.listdir(base_path):
            size_folder_path = os.path.join(base_path, size_folder)
            
            if problem_set == "one_phase_only":
                averages_file = os.path.join(size_folder_path, "averages.json")
                data = load_json_file(averages_file)
                
                for rule in pivot_rules:
                    if rule in data["total_time"]:
                        results[category][problem_set][rule].append(data["total_time"][rule])

            elif problem_set == "two_phases":
                averages_file = os.path.join(size_folder_path, "averages.json")
                data = load_json_file(averages_file)
                
                for rule in pivot_rules:
                    if rule in data["total_time"]:
                        results[category][problem_set][rule].append(data["total_time"][rule])
                
                if category == "balanced_problems":
                    for status in ["Optimal", "Infeasible", "Unbounded"]:
                        status_file = os.path.join(size_folder_path, f"averages_{status}.json")
                        status_data = load_json_file(status_file)
                        
                        for rule in pivot_rules:
                            if rule in status_data.get("total_time", {}):
                                balanced_results[rule][status.lower()].append(status_data["total_time"][rule])
                            
                            # Calculate the percentage of pivot steps in the second phase for Optimal status
                            if status == "Optimal":
                                num_pivot_steps_second_phase = status_data.get("num_pivot_steps_second_phase", {}).get(rule, 0)
                                num_pivot_steps_total = status_data.get("num_pivot_steps_total", {}).get(rule, 0)
                                
                                if num_pivot_steps_total > 0:
                                    percentage_second_phase = (num_pivot_steps_second_phase / num_pivot_steps_total) * 100
                                    second_phase_percentage[rule]["optimal"].append(percentage_second_phase)

# Now, compute the averages for each category, problem set, and pivot rule
final_results = {}

for category, problem_data in results.items():
    final_results[category] = {}
    
    for problem_set, rule_data in problem_data.items():
        final_results[category][problem_set] = {}
        
        for rule, time_data in rule_data.items():
            if len(time_data) > 0:
                final_results[category][problem_set][rule] = sum(time_data) / len(time_data)
            else:
                final_results[category][problem_set][rule] = 0

# Now calculate and display the average time for Infeasible, Unbounded, Optimal for balanced_problems
balanced_avg_times = {}
for rule in pivot_rules:
    balanced_avg_times[rule] = {}
    for status in ["optimal", "infeasible", "unbounded"]:
        if balanced_results[rule][status]:
            balanced_avg_times[rule][status] = sum(balanced_results[rule][status]) / len(balanced_results[rule][status])
        else:
            balanced_avg_times[rule][status] = 0

# Calculate the average percentage of pivot steps in the second phase for each rule (only for Optimal status)
average_second_phase_percentage = {rule: 0 for rule in pivot_rules}
for rule in pivot_rules:
    if second_phase_percentage[rule]["optimal"]:
        average_second_phase_percentage[rule] = sum(second_phase_percentage[rule]["optimal"]) / len(second_phase_percentage[rule]["optimal"])

print("Average times for balanced problems (one-phase and two-phase):")
print(json.dumps(final_results["balanced_problems"], indent=4))

print("\nAverage times for Infeasible/Optimal/Unbounded (balanced problems):")
print(json.dumps(balanced_avg_times, indent=4))

print("\nAverage percentage of pivot steps in the second phase (for Optimal status):")
print(json.dumps(average_second_phase_percentage, indent=4))


Average times for balanced problems (one-phase and two-phase):
{
    "one_phase_only": {
        "dantzig": 3832.708462190628,
        "steepest_edge": 2299.582421922684,
        "random": 7975.263597917557
    },
    "two_phases": {
        "dantzig": 9085.94040608406,
        "steepest_edge": 3248.824888277054,
        "random": 14019.446242904663
    }
}

Average times for Infeasible/Optimal/Unbounded (balanced problems):
{
    "dantzig": {
        "optimal": 10558.922885433416,
        "infeasible": 8013.370296362026,
        "unbounded": 9873.926392396019
    },
    "steepest_edge": {
        "optimal": 3582.097643018281,
        "infeasible": 2772.3060955972264,
        "unbounded": 3635.6348614033654
    },
    "random": {
        "optimal": 16511.236087878002,
        "infeasible": 13196.03009915041,
        "unbounded": 14471.17527281833
    }
}

Average percentage of pivot steps in the second phase (for Optimal status):
{
    "dantzig": 22.738808413715493,
    "steepest_edge"